In [1]:
import random
import numpy as np

In [2]:
map = {}
population = {} #danh sach quan the
populationDistance = {} #danh sach duong di cua moi ca the
fitness = {}
percentFitness = {}
indiNumber = 100 #so ca the trong quan the
const = 25 #so ca the cha me duoc chon va so ca the moi duoc thay the
time = 1000 #so the he

In [3]:
class Data:
    # lay du lieu tu trong file
    def __init__(self, filename):
        lines = open(filename).read().split('\n')
        i = 0
        n = 1
        for line in lines:
            i = i + 1
            if i >= 7 and line != 'EOF' and line != '':
                type1 = [float(_) for _ in line.split()]
                map[n] = type1
                n = n + 1
        print(map)

In [4]:
class GA:
    def cmp(self, a, b):
        return (a > b) - (a < b)

    def DistanceCalculate(self, individual1, individual2):
        # tim ten thanh pho trong bang map va lay toa do cua thanh pho
        for i in map:
            if individual1 == i:
                point1 = map[i]
            if individual2 == i:
                point2 = map[i]
        distance = np.power(((point1[1]-point2[1])**2 + (point1[2]-point2[2])**2), 0.5)
        return distance

    def Distance(self, individual):
        # tinh tong duong di cua ca the
        distance = 0
        for n in range(1, len(individual)+1):
            if n >= 1 and n < len(map):
                p1 = individual[n-1]
                p2 = individual[n]
                distance = distance + self.DistanceCalculate(p1, p2)
            if n == len(map):
                p1 = individual[n-1]
                p2 = individual[0]
                distance = distance + self.DistanceCalculate(p1, p2)
        return distance

    def Check(self, child):
        # kiem tra co phan tu bi trung nhau hay khong
        check = 0
        for i in population:
            if self.cmp(population[i], child) == 0:
                check = check + 1
        return check

    def IntializationIndividual(self):
        # tao ca the
        self.root = list(map.keys())
        random.shuffle(self.root)
        return self.root

    def IntializationPopulation(self):
        # tao quan the
        number = indiNumber
        while number>0:
            population[number] = self.IntializationIndividual()
            populationDistance[number] = self.Distance(population[number])
            number = number - 1
        return population

    def Selection(self):
        # chon lua theo fitness = 1/distance, lua chon ngau nhien boi vong quay roullette
        # luu ti le phan tram tuong ung voi moi diem o percentFitness
        fitnessValue = 0
        for i in populationDistance:
            fitness[i] = float(1/populationDistance[i])
            fitnessValue += float(1/populationDistance[i])
            percentFitness[i] = fitnessValue
        for m in fitness:
            percentFitness[m] = float(percentFitness[m]*100)/fitnessValue
        parent = {}
        i = 1
        while const >= i:
            rand = float(random.randrange(0, 10000))/100
            parent[i] = 1
            for n in percentFitness:
                if percentFitness[n] < rand and percentFitness[n-1] >= rand:
                    parent[i] = n - 1
            out = 0
            for m in parent:
                if parent[m] == parent[i] and m != i:
                    out = 1
            if out == 0:
                i = i + 1
            if out != 0:
                i = i
        return parent

    def Crossover(self, cutRange):
        #lai ghep bang phuong phap doi
        a = 1
        child = {}
        dulp = 0
        #lay phan chung cua parent 1 va parent 2, range [0, cutRange]
        while const >= a:
            ConstRange =[]
            if a == const:
                b = 1
            if a != const:
                b = a+1
            parentID = self.Selection()			
            for i in population:
                if parentID[a] == i:
                    # ConstRange = np.copy(population[i][0:cutRange])
                    for m in range(cutRange):
                        ConstRange.append(population[i][m-1])
            child[a] = ConstRange
            
            #chep phan con lai cua parent 2 vao child 1
            other = 0
            while other < len(map):
                for n in range(len(ConstRange)):
                    if ConstRange[n-1] == population[parentID[b]][other]:
                        dulp = dulp + 1
                if dulp == 0:
                    # child[a] = np.concatenate([child[a], np.array([population[parentID[b]][other]])])
                    child[a].append(population[parentID[b]][other])
                else:
                    dulp = 0
                other = other + 1
            rand1 = float(random.randrange(0, 10000))/10000
            while rand1 <= 0.001:
                self.Mutation(child[a], rand1)
                rand1 = float(random.randrange(0, 10000))/10000
            check = self.Check(child[a])
            if check == 0:
                a = a + 1
            if check != 0:
                a = a
        return child

    def Mutation(self, child, fit):
        if fit <= 0.001:
            print ('Mutant!')
            MutantGen = child[2]
            MutantGen2 = child[5]
            child[2] = MutantGen2
            child[5] = MutantGen
            return child
        else:
            return child

    def Evaluation(self, child):
        r = 1
        worstFitness = {}
        while const >= r:
            # loai bo ca the bo me co fitness nho nhat, thay the bang ca the con
            worstFitness[r] = 0
            for i in range(len(fitness)):
                out = 0
                fitness[0] = 1
                if fitness[i] <= fitness[worstFitness[r]]:
                    for n in range(1, len(worstFitness)+1):
                        if i == worstFitness[n]:
                            out = 1
                    if out == 0:
                        worstFitness[r] = i			
            r = r + 1
        del fitness[0]
        
        r = 1
        while const >= r:
            population[worstFitness[r]] = child[r]
            populationDistance[worstFitness[r]] = self.Distance(child[r])
            r = r + 1
        return population

    def BestIndividual(self):
        bestIndividual = {0: None, 1: None}
        bestDistance = 200000
        pos = 0
        for i in range(1, len(populationDistance)+1):
            # print populationDistance[i]
            if populationDistance[i] <= bestDistance:
                bestDistance = populationDistance[i]
                pos = i
        bestIndividual[0] = population[pos]
        bestIndividual[1] = bestDistance
        return bestIndividual

In [5]:
if __name__ == '__main__':
    print ("File data: ")
    # filename = raw_input ("")
    filename = "a280.tsp"
    a = Data (filename) 
    
    i = 1
    cutRange = 5
    test = GA()
    test.IntializationPopulation()
    BestDistance = test.Distance(population[1])
    while time >= i:
        test.Evaluation(test.Crossover(cutRange))
        print ('Generation',i,'best distance: ', test.Distance(test.BestIndividual()[0]))
        if BestDistance > test.BestIndividual()[1]:
            BestDistance = test.BestIndividual()[1]
            BestWay = test.BestIndividual()[0]
        i = i+1
    print ('population:', population)
    print ('Best way:', BestWay)
    print ('Best distance:', test.Distance(BestWay))

File data: 
{1: [1.0, 288.0, 149.0], 2: [2.0, 288.0, 129.0], 3: [3.0, 270.0, 133.0], 4: [4.0, 256.0, 141.0], 5: [5.0, 256.0, 157.0], 6: [6.0, 246.0, 157.0], 7: [7.0, 236.0, 169.0], 8: [8.0, 228.0, 169.0], 9: [9.0, 228.0, 161.0], 10: [10.0, 220.0, 169.0], 11: [11.0, 212.0, 169.0], 12: [12.0, 204.0, 169.0], 13: [13.0, 196.0, 169.0], 14: [14.0, 188.0, 169.0], 15: [15.0, 196.0, 161.0], 16: [16.0, 188.0, 145.0], 17: [17.0, 172.0, 145.0], 18: [18.0, 164.0, 145.0], 19: [19.0, 156.0, 145.0], 20: [20.0, 148.0, 145.0], 21: [21.0, 140.0, 145.0], 22: [22.0, 148.0, 169.0], 23: [23.0, 164.0, 169.0], 24: [24.0, 172.0, 169.0], 25: [25.0, 156.0, 169.0], 26: [26.0, 140.0, 169.0], 27: [27.0, 132.0, 169.0], 28: [28.0, 124.0, 169.0], 29: [29.0, 116.0, 161.0], 30: [30.0, 104.0, 153.0], 31: [31.0, 104.0, 161.0], 32: [32.0, 104.0, 169.0], 33: [33.0, 90.0, 165.0], 34: [34.0, 80.0, 157.0], 35: [35.0, 64.0, 157.0], 36: [36.0, 64.0, 165.0], 37: [37.0, 56.0, 169.0], 38: [38.0, 56.0, 161.0], 39: [39.0, 56.0, 153.0]

Generation 19 best distance:  30391.69424886978
Generation 20 best distance:  30391.69424886978
Generation 21 best distance:  30391.69424886978
Generation 22 best distance:  30391.69424886978
Generation 23 best distance:  30368.23371065792
Generation 24 best distance:  30368.23371065792
Generation 25 best distance:  30368.23371065792
Generation 26 best distance:  30211.069981968532
Generation 27 best distance:  30211.069981968532
Generation 28 best distance:  30211.069981968532
Generation 29 best distance:  30211.069981968532
Generation 30 best distance:  30211.069981968532
Generation 31 best distance:  30211.069981968532
Generation 32 best distance:  30211.069981968532
Generation 33 best distance:  30118.680119918725
Generation 34 best distance:  30118.680119918725
Generation 35 best distance:  30104.78290206611
Mutant!
Generation 36 best distance:  30104.78290206611
Generation 37 best distance:  30099.63105152038
Generation 38 best distance:  30099.63105152038
Generation 39 best dist

Generation 185 best distance:  28114.061296354295
Generation 186 best distance:  28085.43125616182
Generation 187 best distance:  28085.43125616182
Mutant!
Generation 188 best distance:  28085.43125616182
Generation 189 best distance:  28085.43125616182
Generation 190 best distance:  28085.43125616182
Generation 191 best distance:  28085.43125616182
Generation 192 best distance:  28058.679817318254
Generation 193 best distance:  28034.816332073533
Generation 194 best distance:  28034.816332073533
Generation 195 best distance:  28034.816332073533
Generation 196 best distance:  28034.816332073533
Generation 197 best distance:  28034.816332073533
Generation 198 best distance:  28034.816332073533
Generation 199 best distance:  28034.816332073533
Generation 200 best distance:  28011.719558719687
Generation 201 best distance:  28011.719558719687
Generation 202 best distance:  28005.685680381204
Generation 203 best distance:  27984.431237251814
Generation 204 best distance:  27984.43123725181

Generation 350 best distance:  26404.019187162292
Generation 351 best distance:  26404.019187162292
Generation 352 best distance:  26404.019187162292
Generation 353 best distance:  26404.019187162292
Generation 354 best distance:  26404.019187162292
Generation 355 best distance:  26404.019187162292
Generation 356 best distance:  26404.019187162292
Generation 357 best distance:  26404.019187162292
Generation 358 best distance:  26404.019187162292
Generation 359 best distance:  26404.019187162292
Generation 360 best distance:  26389.79960094786
Generation 361 best distance:  26389.79960094786
Generation 362 best distance:  26389.79960094786
Generation 363 best distance:  26389.79960094786
Generation 364 best distance:  26382.176972691177
Generation 365 best distance:  26382.176972691177
Generation 366 best distance:  26382.176972691177
Generation 367 best distance:  26248.22487649262
Generation 368 best distance:  26248.22487649262
Generation 369 best distance:  26248.22487649262
Generat

Generation 515 best distance:  24875.261186511103
Generation 516 best distance:  24875.261186511103
Generation 517 best distance:  24875.261186511103
Generation 518 best distance:  24770.87817152364
Generation 519 best distance:  24770.87817152364
Generation 520 best distance:  24770.87817152364
Generation 521 best distance:  24770.87817152364
Generation 522 best distance:  24770.87817152364
Generation 523 best distance:  24770.87817152364
Generation 524 best distance:  24770.87817152364
Generation 525 best distance:  24736.988085782097
Generation 526 best distance:  24736.988085782097
Generation 527 best distance:  24736.988085782097
Generation 528 best distance:  24736.988085782097
Generation 529 best distance:  24734.531073208218
Generation 530 best distance:  24734.531073208218
Generation 531 best distance:  24681.663204276585
Generation 532 best distance:  24681.663204276585
Generation 533 best distance:  24681.663204276585
Generation 534 best distance:  24681.663204276585
Generat

Generation 680 best distance:  23311.138758918947
Generation 681 best distance:  23311.138758918947
Generation 682 best distance:  23311.138758918947
Generation 683 best distance:  23311.138758918947
Generation 684 best distance:  23282.330281315943
Generation 685 best distance:  23282.330281315943
Generation 686 best distance:  23282.330281315943
Generation 687 best distance:  23282.330281315943
Generation 688 best distance:  23282.330281315943
Generation 689 best distance:  23282.330281315943
Generation 690 best distance:  23282.330281315943
Generation 691 best distance:  23168.120972146833
Generation 692 best distance:  23168.120972146833
Generation 693 best distance:  23160.16506685818
Generation 694 best distance:  23160.16506685818
Generation 695 best distance:  23160.16506685818
Generation 696 best distance:  23160.16506685818
Mutant!
Generation 697 best distance:  23160.16506685818
Generation 698 best distance:  23160.16506685818
Generation 699 best distance:  23160.16506685818

Generation 845 best distance:  21293.934700328817
Generation 846 best distance:  21293.934700328817
Generation 847 best distance:  21293.934700328817
Generation 848 best distance:  21293.934700328817
Generation 849 best distance:  21293.934700328817
Generation 850 best distance:  21273.662534421765
Generation 851 best distance:  21273.662534421765
Generation 852 best distance:  21246.401291991977
Generation 853 best distance:  21246.401291991977
Generation 854 best distance:  21246.401291991977
Generation 855 best distance:  21246.401291991977
Generation 856 best distance:  21169.678087347173
Generation 857 best distance:  21117.9430227345
Generation 858 best distance:  21117.9430227345
Generation 859 best distance:  21112.619151738043
Generation 860 best distance:  21090.180158253315
Generation 861 best distance:  21090.180158253315
Generation 862 best distance:  21090.180158253315
Generation 863 best distance:  21090.180158253315
Generation 864 best distance:  21090.180158253315
Gene